# Data

## Taxifare

In [2]:
import pandas as pd

def get_data():
    url = "s3://wagon-public-datasets/taxi-fare-train.csv"
    df = pd.read_csv(url, nrows=100)
    return df

In [4]:
df = get_data()
df

,key,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
0,2009-06-15 17:26:21.0000001,4.5,2009-06-15 17:26:21 UTC,-73.844311,40.721319,-73.841610,40.712278,1
1,2010-01-05 16:52:16.0000002,16.9,2010-01-05 16:52:16 UTC,-74.016048,40.711303,-73.979268,40.782004,1
2,2011-08-18 00:35:00.00000049,5.7,2011-08-18 00:35:00 UTC,-73.982738,40.761270,-73.991242,40.750562,2
3,2012-04-21 04:30:42.0000001,7.7,2012-04-21 04:30:42 UTC,-73.987130,40.733143,-73.991567,40.758092,1
4,2010-03-09 07:51:00.000000135,5.3,2010-03-09 07:51:00 UTC,-73.968095,40.768008,-73.956655,40.783762,1
...,...,...,...,...,...,...,...,...
95,2013-07-06 17:18:00.000000119,13.5,2013-07-06 17:18:00 UTC,-73.950655,40.783282,-73.984365,40.769802,1
96,2013-10-08 17:29:00.00000025,7.5,2013-10-08 17:29:00 UTC,-73.976512,40.765277,-73.977672,40.778052,1
97,2014-12-07 12:26:00.0000009,9.0,2014-12-07 12:26:00 UTC,-73.984977,40.752122,-74.000925,40.757982,1
98,2015-05-17 08:02:59.0000002,14.0,2015-05-17 08:02:59 UTC,-73.979210,40.723682,-74.017464,40.711395,1


## Clean data

In [5]:
def clean_df(df):
    df = df.dropna(how='any', axis='rows')
    df = df[(df.dropoff_latitude != 0) | (df.dropoff_longitude != 0)]
    df = df[(df.pickup_latitude != 0) | (df.pickup_longitude != 0)]
    if "fare_amount" in list(df):
        df = df[df.fare_amount.between(0, 4000)]
    df = df[df.passenger_count < 8]
    df = df[df.passenger_count >= 1]
    df = df[df["pickup_latitude"].between(left=40, right=42)]
    df = df[df["pickup_longitude"].between(left=-74.3, right=-72.9)]
    df = df[df["dropoff_latitude"].between(left=40, right=42)]
    df = df[df["dropoff_longitude"].between(left=-74, right=-72.9)]
    return df

In [6]:
df = get_data()

In [7]:
df.head(3)

,key,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
0,2009-06-15 17:26:21.0000001,4.5,2009-06-15 17:26:21 UTC,-73.844311,40.721319,-73.841610,40.712278,1
1,2010-01-05 16:52:16.0000002,16.9,2010-01-05 16:52:16 UTC,-74.016048,40.711303,-73.979268,40.782004,1
2,2011-08-18 00:35:00.00000049,5.7,2011-08-18 00:35:00 UTC,-73.982738,40.761270,-73.991242,40.750562,2


In [8]:
df.shape

(100, 8)

In [9]:
df = clean_df(df)

In [10]:
df.shape

(84, 8)

# Model

In [11]:
from sklearn.ensemble import RandomForestRegressor

model_params = dict(
  n_estimators=100,
  max_depth=1)

model = RandomForestRegressor()
model.set_params(**model_params)

RandomForestRegressor(max_depth=1)

# Holdout

In [12]:
from sklearn.model_selection import train_test_split

y_train = df["fare_amount"]
X_train = df.drop("fare_amount", axis=1)

X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.1)

# Transformers

In [13]:
def minkowski_distance(df, p,
                       start_lat="pickup_latitude",
                       start_lon="pickup_longitude",
                       end_lat="dropoff_latitude",
                       end_lon="dropoff_longitude"):
    x1 = df[start_lon]
    x2 = df[end_lon]
    y1 = df[start_lat]
    y2 = df[end_lat]
    return ((abs(x2 - x1) ** p) + (abs(y2 - y1)) ** p) ** (1 / p)

In [14]:
from sklearn.base import BaseEstimator, TransformerMixin

class DistanceTransformer(BaseEstimator, TransformerMixin):

    def __init__(self, distance_type="euclidian", **kwargs):
        self.distance_type = distance_type

    def transform(self, X, y=None):
        assert isinstance(X, pd.DataFrame)
        if self.distance_type == "haversine":
            X["distance"] = haversine_vectorized(X)
        if self.distance_type == "euclidian":
            X["distance"] = minkowski_distance(X, p=2)
        if self.distance_type == "manhattan":
            X["distance"] = minkowski_distance(X, p=1)
        return X[["distance"]]

    def fit(self, X, y=None):
        return self

# Pipeline

In [15]:
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer

pipe_distance = make_pipeline(
    DistanceTransformer(),
    StandardScaler())


cols = ["pickup_latitude", "pickup_longitude", "dropoff_latitude", "dropoff_longitude"]

feateng_blocks = [
    ('distance', pipe_distance, cols),
]

features_encoder = ColumnTransformer(feateng_blocks)

pipeline = Pipeline(steps=[
            ('features', features_encoder),
            ('model', model)])

In [16]:
pipeline.fit(X_train, y_train)

Pipeline(steps=[('features',
                 ColumnTransformer(transformers=[('distance',
                                                  Pipeline(steps=[('distancetransformer',
                                                                   DistanceTransformer()),
                                                                  ('standardscaler',
                                                                   StandardScaler())]),
                                                  ['pickup_latitude',
                                                   'pickup_longitude',
                                                   'dropoff_latitude',
                                                   'dropoff_longitude'])])),
                ('model', RandomForestRegressor(max_depth=1))])

# Metrics

In [17]:
import numpy as np

def compute_rmse(y_pred, y_true):
    return np.sqrt(((y_pred - y_true) ** 2).mean())

In [18]:
y_pred = pipeline.predict(X_test)

In [19]:
rmse = compute_rmse(y_pred, y_test)

In [20]:
rmse

2.239487079873548

# MLflow

In [18]:
from memoized_property import memoized_property

import mlflow
from  mlflow.tracking import MlflowClient

class MLFlowBase():

    def __init__(self, experiment_name, MLFLOW_URI):
        self.experiment_name = experiment_name
        self.MLFLOW_URI = MLFLOW_URI

    @memoized_property
    def mlflow_client(self):
        mlflow.set_tracking_uri(self.MLFLOW_URI)
        return MlflowClient()

    @memoized_property
    def mlflow_experiment_id(self):
        try:
            return self.mlflow_client \
                .create_experiment(self.experiment_name)
        except BaseException:
            return self.mlflow_client \
                .get_experiment_by_name(self.experiment_name).experiment_id

    def mlflow_create_run(self):
        self.mlflow_run = self.mlflow_client \
            .create_run(self.mlflow_experiment_id)

    def mlflow_log_param(self, key, value):
        self.mlflow_client \
            .log_param(self.mlflow_run.info.run_id, key, value)

    def mlflow_log_metric(self, key, value):
        self.mlflow_client \
            .log_metric(self.mlflow_run.info.run_id, key, value)


In [19]:
pipeline.get_params()

/Users/gmanchon/.pyenv/versions/3.8.5/envs/data_502/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


{'memory': None,
 'steps': [('features',
   ColumnTransformer(transformers=[('distance',
                                    Pipeline(steps=[('distancetransformer',
                                                     DistanceTransformer()),
                                                    ('standardscaler',
                                                     StandardScaler())]),
                                    ['pickup_latitude', 'pickup_longitude',
                                     'dropoff_latitude', 'dropoff_longitude'])])),
  ('model', RandomForestRegressor(max_depth=1))],
 'verbose': False,
 'features': ColumnTransformer(transformers=[('distance',
                                  Pipeline(steps=[('distancetransformer',
                                                   DistanceTransformer()),
                                                  ('standardscaler',
                                                   StandardScaler())]),
                                  ['p

In [20]:
from sklearn.model_selection import GridSearchCV

grid_search = GridSearchCV(
    pipeline, 
    param_grid={
        'features__distance__standardscaler__copy': [True],
        'model__min_samples_leaf': [3],
        'model__oob_score': [True],
        'model__min_weight_fraction_leaf': [0.0, 0.1]
    },
    cv=5
)

grid_search.fit(X_train, y_train)
grid_search.score(X_test, y_test)

grid_search.best_estimator_
grid_search.best_params_

{'features__distance__standardscaler__copy': True,
 'model__min_samples_leaf': 3,
 'model__min_weight_fraction_leaf': 0.1,
 'model__oob_score': True}